In [7]:
from sentence_transformers import SentenceTransformer
import os, glob
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string as st
import pickle

In [8]:
PATH_TO_DOCUMENTS = './/data//plaintext_articles//'
document_mapper = pd.DataFrame(os.listdir('.//data//plaintext_articles'),columns=['file_name'])

In [9]:
def load_file(filename: str) -> str:
    with open(f"{PATH_TO_DOCUMENTS}{filename}", "r", encoding="utf-8") as f:
        return f.read()
    
document_mapper["docs"] = document_mapper["file_name"].apply(load_file)

#removing ".txt" from file_name
document_mapper["file_name"] = document_mapper["file_name"].apply(lambda x : x[:-4])

In [40]:
def clean_text(document : str) -> str :
    tokens = word_tokenize(document)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word
    table = str.maketrans('', '', st.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in stripped if not w in stop_words]
    return (" ").join(words)

In [122]:
document_mapper['clean_text'] = document_mapper['docs'].apply(clean_text)

In [149]:
encoding_model = SentenceTransformer('all-MiniLM-L12-v2')
encoding_model.max_seq_length = 512
print("Max Sequence Length:", encoding_model.max_seq_length)

Max Sequence Length: 512


In [152]:
embeddings = encoding_model.encode(document_mapper['clean_text'].tolist(),show_progress_bar=True)
document_mapper['embeddings'] = embeddings.tolist()
document_mapper.to_pickle('embeddings.pkl')

Batches:   0%|          | 0/144 [00:00<?, ?it/s]